This is a class for a graph representation of a lattice.
Basic structure is to have three lists: an index list (just a range), an adgacency list which is a list of lists, and a vertex list, which contains the value (1 or -1) at each node. 

In [103]:
import random
import pandas as pd
class graph():
    
    def __init__(self, J,):
        """
        Init method for a graph calss.
        creates a graph object to represent an arbitrary lattice of 2 or 3 dimensions (only 2D currently implemented)
        @param: J: float
            this is the J value of the lattice model we want to simulate. in physics, this is a property of the material
        """
        self.J = J
        self.vertex = []
        self.adj = []

    def set_J(self, J):
        """
        method to set or change the J value for an ising graph model
        side effect: updates self.J
        """
        self.J = J
    
    

    def initialize(self, lattice, dimension = 2):
        """
        method to build/reset the actual structure of the graph
        @param = lattice: DataFrame
            - lattice is a dataframe representing the physical structure of the ising model. It can be is 2
            or 3 dimensions
        @param: dimension: int (2 or 3)
            tells the method whether to interpret the lattice as 2D or 3D
        """
        # for 2D:
        M, N = lattice.shape
        self.adj = adjacency_list(lattice, dimension)
        self.vertex = lattice.values.flatten()

    def flip(self, nodes):
        """
        Method to flip one or more nodes/vertices of the graph
        @param: nodes: This is either a single integer or a list of integers, that are the index of the nodes to be flipped
        side effects: flips nodes
        returns: none
        """
        if isinstance(nodes, int):
            self.vertex[nodes] = -1*self.vertex[nodes]
        else:
            for node in nodes:
                self.vertex[nodes] = -1*self.vertex[nodes]

    def getE(self, nodes,):
        """
        Method to find the energy associated with a node or a group of nodes (in the case of a cluster)
        @param: nodes: This is either a single integer or a list of integers, that are the index of the nodes to be flipped
        side effects: none
        returns: energy associated with node/nodes
        """
        sum = 0 #initialize energy E
        if isinstance(nodes, int): #if just one node
            node = nodes
            for neighbor in self.adj[node]:
                sum += self.vertex[neighbor] * self.vertex[node]

        
        else: #if multiple nodes
            completed_pairs = [] #keep track of pairs that are already checked
            for node in nodes:
                for neighbor in self.adj(node):
                    pair = [node, neighbor]
                    pair_inv = [neighbor, node]
                    if pair not in completed_pairs or pair_inv not in completed_pairs:
                        sum += self.vertex[node] * self.vertex[neighbor]
                        completed_pairs.append(pair)
        E = self.J*(-1)*(sum)
        return E
    
    def randomize_all(self):
        for node in self.vertex:
            if self.vertex[node] != 0:
                node = random.choice([-1, 1])
    def getM(self):
        sum = 0
        for vertex in self.vertex:
            sum += self.vertex
        return sum

def flattened_index(coords, N):
        """
        function to return the flattened index of a point at i, j
        @param: coords: should be list of two elements: i, j, which are the index of the location in a 2d dataframe
        """
        i = coords[0]
        j = coords[1]
        return i*N + j
def adjacency_list(lattice, dimension):
        """
        function to build an adjacency out of the lattice. used as part of the graph initialize method.
        returns an M*N dataframe, where the values are lists of the flattened index of the adjacent nodes
        """
        M, N = lattice.shape
        empty_frame = [[[] for _ in range(M)] for _ in range(N)]
        matrix  = pd.DataFrame(empty_frame)
        for i in range(M):
            for j in range(N):
                neighbors = []
                if i > 0:
                    neighbors.append(flattened_index([i-1,j], N))
                if i < (M-2):
                    neighbors.append(flattened_index([i+1,j], N))
                if j > 0:
                    neighbors.append(flattened_index([i,j-1], N))
                if j < (N-2):
                    neighbors.append(flattened_index([i,j+1], N))
                
                matrix.at[i, j] = neighbors
        adjacency_list = matrix.values.flatten()
        return adjacency_list


below is some testing code:

In [104]:
import pandas as pd

# Create a sample DataFrame
data = {'A': [1, 2, 3],
        'B': [4, 5, 6],
        'C': [7, 8, 9]}
df = pd.DataFrame(data)
print(df)
# Flatten the DataFrame
flattened_array = df.values.flatten()

print("Flattened DataFrame as 1D array:")
print(flattened_array)

M, N = (4,4)
data = [[[_, _+1] for _ in range(M)] for _ in range(N)]
matrix  = pd.DataFrame(data)
print(matrix)
print(matrix.values.flatten())



   A  B  C
0  1  4  7
1  2  5  8
2  3  6  9
Flattened DataFrame as 1D array:
[1 4 7 2 5 8 3 6 9]
        0       1       2       3
0  [0, 1]  [1, 2]  [2, 3]  [3, 4]
1  [0, 1]  [1, 2]  [2, 3]  [3, 4]
2  [0, 1]  [1, 2]  [2, 3]  [3, 4]
3  [0, 1]  [1, 2]  [2, 3]  [3, 4]
[list([0, 1]) list([1, 2]) list([2, 3]) list([3, 4]) list([0, 1])
 list([1, 2]) list([2, 3]) list([3, 4]) list([0, 1]) list([1, 2])
 list([2, 3]) list([3, 4]) list([0, 1]) list([1, 2]) list([2, 3])
 list([3, 4])]


In [105]:
# Create a sample DataFrame
data = {'A': [1, 2, 3],
        'B': [4, 5, 6],
        'C': [7, 8, 9]}
df = pd.DataFrame(data)

test_graph = graph(1)
test_graph.initialize(df)

Testing by building off of lattice class! 

In [106]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numba
from numba import njit
from scipy.ndimage import convolve, generate_binary_structure


In [107]:
class Lattice:
    size = 0
    shape = False
    temp = 0
    
    def __init__(self, J, size = 10, shape = False, rand = True):
        self.J = J
        #initialize lattice grid --> 10x10 square grid filled w zeros
        data = [[0 for _ in range(size)] for _ in range(size)]
        self.size = size
        self.shape = shape
        self.lat = pd.DataFrame(data)
            
        for i in range(size):
            j_range = size
            if (shape == True):
                j_range = size - i
            for j in range(j_range):
                if (rand == False):
                    self.lat.at[i, j] = 1
                if (rand == True):
                    num = random.random()
                    if (num < 0.5):
                        self.lat.at[i, j] = -1
                    else:
                        self.lat.at[i, j] = 1
                        
    #change spin (takes in a list of coordinates)
    def flip(self, lat_list):
        if type(lat_list) is not list:
            if self.lat.at[lat_list[0], lat_list[1]] != 0:
                self.lat.at[lat_list[0], lat_list[1]] = -1*(self.lat.at[lat_list[0], lat_list[1]])
        else:
            for i in range(len(lat_list)):
                if self.lat.at[lat_list[i][0], lat_list[i][1]] != 0:
                    self.lat.at[lat_list[i][0], lat_list[i][1]] = -1*(self.lat.at[lat_list[i][0], lat_list[i][1]])

    def getE(self):
        return 0
    
    def get_lattice(self):
        return self.lat
    
    def set_temp(self, T):
        self.temp = T
    
    def num_lattice(self):
        print(self.lat.to_string(header=False, index=False))
    
    def color_lattice(self):
        plt.clf()
        plt.imshow(self.lat)

In [115]:
square_lattice = Lattice(J = 1)
square_df = square_lattice.get_lattice()
square_graph = graph(1)
print("check")
square_graph.initialize(square_df)
print(square_graph.getE(0))
print(square_graph.vertex)
square_graph.flip([0, 25, 99])
print(square_graph.vertex)
print(square_graph.getE(0))

check
-2
[ 1  1  1 -1 -1  1  1  1 -1  1  1  1  1  1  1  1  1  1 -1  1  1  1 -1  1
 -1 -1  1 -1  1 -1 -1 -1  1  1 -1  1  1 -1  1  1  1  1 -1 -1  1  1  1  1
 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1  1  1  1  1  1 -1  1  1  1  1  1
 -1 -1 -1  1  1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  1  1
 -1  1  1 -1]
[-1  1  1 -1 -1  1  1  1 -1  1  1  1  1  1  1  1  1  1 -1  1  1  1 -1  1
 -1  1  1 -1  1 -1 -1 -1  1  1 -1  1  1 -1  1  1  1  1 -1 -1  1  1  1  1
 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1  1  1  1  1  1 -1  1  1  1  1  1
 -1 -1 -1  1  1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  1  1
 -1  1  1  1]
2
